In [121]:
import pandas as pd 
import json
import sentifish
import requests
import urllib
import pprint
import google_play_scraper
from sentifish import Sentiment
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all
from google_play_scraper import permissions

In [122]:
with open('Data.jl') as f:
    lines = f.read().splitlines()

In [123]:
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [124]:
print(df_inter.columns)

Index(['json_element'], dtype='object')


In [125]:

df_inter['json_element'].apply(json.loads)

0       {'appId': 'com.casualgamefactory.wordapart', '...
1       {'appId': 'com.noodlecake.typeshift', 'appName...
2       {'appId': 'com.q42.ohno', 'appName': '0h n0', ...
3       {'appId': 'com.finalgamestudio.aurora.hex.jigs...
4       {'appId': 'za.co.twinc.a9letterjumble', 'appNa...
                              ...                        
9995    {'appId': 'com.simulationcurriculum.skysafari5...
9996    {'appId': 'com.vitotechnology.StarWalk', 'appN...
9997    {'appId': 'com.pa.lightpollutionmap', 'appName...
9998    {'appId': 'com.escapistgames.starchartgoogleed...
9999    {'appId': 'com.cgs.HiddenSurvivor', 'appName':...
Name: json_element, Length: 10000, dtype: object

In [126]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

## Frequency of type of permission in Android Apps

In [127]:
permissiontypeFrequencyCount=[]
typeaccess=[]

In [128]:
for typecount in range(len(df_final)):
    #print(len(df_final['allPermissions'][typecount]))
    for item in range(len(df_final['allPermissions'][typecount])):
        #print(df_final['allPermissions'][typecount][item]['type'])
        if df_final['allPermissions'][typecount][item]['type'] not in permissiontypeFrequencyCount:
            permissiontypeFrequencyCount.append(df_final['allPermissions'][typecount][item]['type'])
for elem in permissiontypeFrequencyCount:
    if elem not in typeaccess:
        typeaccess.append(elem)
print(typeaccess)

['Photos/Media/Files', 'Storage', 'Wi-Fi connection information', 'Other', 'Phone', 'Device ID & call information', 'Location', 'Device & app history', 'Camera', 'Microphone', 'Identity', 'Contacts', 'Calendar', 'Cellular data settings', 'SMS', 'Wearable sensors/Activity data']


In [129]:
print(len(typeaccess))
typeaccessDict={}
for keyintype in typeaccess:
    typeaccessDict[keyintype]=0
pprint.pprint(typeaccessDict)


16
{'Calendar': 0,
 'Camera': 0,
 'Cellular data settings': 0,
 'Contacts': 0,
 'Device & app history': 0,
 'Device ID & call information': 0,
 'Identity': 0,
 'Location': 0,
 'Microphone': 0,
 'Other': 0,
 'Phone': 0,
 'Photos/Media/Files': 0,
 'SMS': 0,
 'Storage': 0,
 'Wearable sensors/Activity data': 0,
 'Wi-Fi connection information': 0}


In [130]:
pprint.pprint(df_final['allPermissions'][0])

[{'permission': 'read the contents of your USB storage',
  'type': 'Photos/Media/Files'},
 {'permission': 'modify or delete the contents of your USB storage',
  'type': 'Photos/Media/Files'},
 {'permission': 'read the contents of your USB storage', 'type': 'Storage'},
 {'permission': 'modify or delete the contents of your USB storage',
  'type': 'Storage'},
 {'permission': 'view Wi-Fi connections',
  'type': 'Wi-Fi connection information'},
 {'permission': 'view network connections', 'type': 'Other'},
 {'permission': 'full network access', 'type': 'Other'},
 {'permission': 'run at startup', 'type': 'Other'},
 {'permission': 'control vibration', 'type': 'Other'},
 {'permission': 'prevent device from sleeping', 'type': 'Other'}]


In [131]:
for typecount in range(len(df_final)):
    #print(len(df_final['allPermissions'][typecount]))
    count=[]
    avoiddup=[]
    for item in range(len(df_final['allPermissions'][typecount])):
        if df_final['allPermissions'][typecount][item]['type'] in typeaccessDict.keys():
            count.append(df_final['allPermissions'][typecount][item]['type'])
    #print(count)
    for elem in count:
        if elem not in avoiddup:
            avoiddup.append(elem)
    #print(avoiddup)
    for k in avoiddup:
        if k in typeaccessDict.keys():
            temp=typeaccessDict[k]
            typeaccessDict[k]=temp+1

    

In [149]:
totalC=0
print ("{:<15} {:<15} ".format('PERMISSION-TYPE', 'APP-COUNT'))
for key, value in typeaccessDict.items():
    totalC+=value
for key, value in typeaccessDict.items():
    print ("{:<15} {:<15} ".format(key,(value/totalC)*100))
print(totalC)

PERMISSION-TYPE APP-COUNT       
Photos/Media/Files 15.500730052391996 
Storage         15.496435626556728 
Wi-Fi connection information 10.869191789057803 
Other           21.05556987030834 
Phone           5.0094477368375845 
Device ID & call information 4.65945203126342 
Location        5.898393884737611 
Device & app history 2.3168427381259127 
Camera          6.22262303530018 
Microphone      3.5493429528472045 
Identity        3.5407541011766726 
Contacts        4.556385811217041 
Calendar        0.9297431933350512 
Cellular data settings 0.049385897105556985 
SMS             0.2555183371983166 
Wearable sensors/Activity data 0.09018294254058232 
46572


# Zero Permission Apps Case Study

In [46]:
#Zero Permission Apps from the take dataset
zeroPermissionappList=[]
for zeroApp in range(len(df_final)):
    if len(df_final['allPermissions'][zeroApp])==0:
        zeroPermissionappList.append(df_final['appId'][zeroApp])
print(zeroPermissionappList)
#This zero permission app that link to an external source or fake privacy policy permission

['com.hyperionics.avarLic', 'org.coolreader.donation.silver', 'com.Hololive_Stickers', 'com.ginkage.ejlookup', 'com.joseluisgalan.android.chatstats', 'info.casualcomputing.sputter', 'com.piomusic.piomusicv', 'de.smartchord.droid.plug.cof', 'com.bsoft.asadkhan.maqamatalarabiya', 'online.astrotools.miroir3', 'com.rmsoft.myemdr', 'com.vlvolad.quantumoscillator', 'com.cranf.android.deepware.rupture', 'com.battery.lifespan.extender', 'com.cranf.android.quickonf.battery.free', 'southbridge.led_calculatorpro', 'jackpal.droidexaminer', 'com.scappyapps.fourthirtytwo', 'com.zebproj.etherpad', 'com.iforpowell.android.ipbikekey', 'de.khe.ringrechner', 'com.MikaMobile.Battleheart', 'ru.wellapp.bukvica', 'com.simplesoftwarestudio.troparions', 'com.simplesoftwarestudio.molitvoslov', 'com.medclever.pocketdoctor_free', 'com.simplesoftwarestudio.psaltyr', 'com.ovirium.simoron', 'com.pasanen.belarusbirds', 'com.nappsislamic.znaniia', 'com.faultexception.reader.pro', 'com.Waseem.muhammadusama.GeoEngineeri

In [49]:
print(len(zeroPermissionappList))
zeroPermissionappDict={}

for app_id in zeroPermissionappList:
    try:
        getCategory = app(
        app_id,
        lang='en', # defaults to 'en'
        country='us' # defaults to 'us'
        )
        

        key=getCategory['title']
        value=getCategory['genre']
        print(key+"-"+value)
        #create a key value pair of app Category and app Name
        zeroPermissionappDict.update({key:value})
    except urllib.error.HTTPError:
        continue
    except google_play_scraper.exceptions.NotFoundError:
        continue
    except AttributeError:
        continue
    except ValueError:
        continue
    except requests.exceptions.HTTPError:
        continue
    except google_play_scraper.exceptions.ExtraHTTPError:
        continue

    

124
@Voice Premium License-Productivity
Cool Reader Silver Donation-Tools
Hololive Stickers-Communication
EJLookup — Japanese Dictionary-Books & Reference
Chat Stats for WhatsApp-Social
Sputter Music Sequencer-Music & Audio
PiOMuSiC-Education
s.mart Circle of Fifths-Books & Reference
Maqamat Al-Arabiya-Music & Audio
COMPLETE ASTROLOGICAL PORTRAIT (an Astral Mirror)-Lifestyle
My EMDR-Lifestyle
Quantum Harmonic Oscillator-Education
Deepware Rupture-Productivity
Battery Lifespan Extender-Tools
QuickONF Battery Widgets-Tools
LED Calculator Pro-Tools
Droid Examiner-Tools
Binaural Chakra Therapy-Health & Fitness
EtherSurface-Music & Audio
IpBikeKey-Health & Fitness
Shooting-Score-Calculator-Sports
Battleheart-Role Playing
Буквица древних Славян-Education
Тропари православные-Books & Reference
Молитвослов-Books & Reference
Первая помощь - Карманный доктор (базовая версия)-Medical
Псалтирь-Books & Reference
Симорон, Ритуалы, Желания-Lifestyle
Птицы Беларуси-Education
ОБЯЗАТЕЛЬНЫЕ ЗНАНИЯ ДЛЯ МУ

In [ ]:
# collect zero Permission app from Social and Communcation as my study......
socialAppList=[]
communicationAppList=[]
for name,category in zeroPermissionappDict.items():
    if(category=='Communication'):
        communicationAppList.append(name)
    if(category=='Social'):
        socialAppList.append(name)


### Black List Apps From Zero Permissions..

In [64]:
print("*****************Communication Apps**************")
for i in communicationAppList:
    print(i)
print("*****************Socail Apps*********************")
for j in socialAppList:
    print(j)

*****************Communication Apps**************
Hololive Stickers
ANT USB Service
*****************Socail Apps*********************
Chat Stats for WhatsApp


# Most Dangerous Permission Apps Case Study

In [25]:
Dangerous=['directly call phone numbers','read call log','read phone status and identity','read sensitive log data','retrieve running apps','delete all app cache data']
appList=[]
temp=[]
permissionList=[]
#Generate the list of apps that access all the above list of  dangerous permissions

for i in range(len(df_final)):#from .jl jsonlines 
    #print(df_final['appName'][i])
    temp=df_final['allPermissions'][i]
    #print(temp)
    for j in range(len(temp)):
        #print(temp[j]['permission'],j)
        permissionList.append(temp[j]['permission'])
    if Dangerous[0] in permissionList and Dangerous[1] in permissionList and Dangerous[2] in permissionList and Dangerous[3] in permissionList and Dangerous[4] in permissionList:
        appList.append(df_final['appId'][i])
        permissionList=[]
        temp=[]
    else:
        permissionList=[]
        temp=[]
#print(appList)
new_appList = []
for elem in appList:
    if elem not in new_appList:
        new_appList.append(elem) 
print(new_appList)

['ru.agc.acontactnexttrial', 'net.soti.mobicontrol.androidwork', 'net.dinglisch.android.taskerm', 'com.huawei.health']


In [27]:
dangerousDict={}
for app_id in new_appList:
    getCategory = app(
    app_id,
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
    )
    key=getCategory['title']
    value=getCategory['genre']
    dangerousDict.update({key:value})

In [44]:
 pprint.pprint(dangerousDict)

{'Huawei Health': 'Health & Fitness',
 'MobiControl | Android Enterprise': 'Business',
 'Phone and Contacts - AGContacts, Lite edition': 'Communication',
 'Tasker': 'Tools'}


In [45]:
permissionFullList=[]
for app_id in new_appList:
    result = permissions(
    app_id,
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    )
    permissionFullList.append(result)   

In [105]:
pprint.pprint(permissionFullList[0])

{'Calendar': ['read calendar events plus confidential information'],
 'Contacts': ['find accounts on the device',
              'modify your contacts',
              'read your contacts'],
 'Device & app history': ['read sensitive log data', 'retrieve running apps'],
 'Device ID & call information': ['read phone status and identity'],
 'Identity': ['add or remove accounts',
              'find accounts on the device',
              'modify your own contact card',
              'read your own contact card'],
 'Microphone': ['record audio'],
 'Other': ['change your audio settings',
           'control Near Field Communication',
           'control vibration',
           'full network access',
           'install shortcuts',
           'prevent device from sleeping',
           'read sync settings',
           'run at startup',
           'send sticky broadcast',
           'toggle sync on and off',
           'use accounts on the device',
           'view network connections'],
 'Phone':

In [134]:
for key,value in dangerousDict.items():
    print("\033[1m APP NAME : \033[0m "+key)
    print("\n")
for index in permissionFullList:
    totalCount=0
    print("\033[1m Name    ","Count  \033[0m")
    for i,j in index.items():
        totalCount+=len(j)
        print (i+" "+str(len(j)))
    print("\033[1m Total Permission:\033[0m",totalCount)
    print("\n")

 APP NAME :  Phone and Contacts - AGContacts, Lite edition


 APP NAME :  MobiControl | Android Enterprise


 APP NAME :  Tasker


 APP NAME :  Huawei Health


 Name     Count  
Phone 5
Photos/Media/Files 2
Contacts 3
Device & app history 2
Identity 4
Calendar 1
Storage 2
Microphone 1
Wi-Fi connection information 1
Device ID & call information 1
Other 12
Uncategorized 1
 Total Permission: 35


 Name     Count  
Wi-Fi connection information 1
Cellular data settings 1
Photos/Media/Files 4
Device ID & call information 1
Phone 5
Location 1
Device & app history 2
Contacts 3
Identity 3
Camera 1
Microphone 1
Storage 2
Other 29
Uncategorized 37
 Total Permission: 91


 Name     Count  
Phone 6
Photos/Media/Files 2
SMS 4
Device ID & call information 1
Contacts 2
Storage 2
Wi-Fi connection information 1
Microphone 1
Wearable sensors/Activity data 1
Camera 1
Calendar 2
Device & app history 3
Location 2
Other 25
Uncategorized 7
 Total Permission: 60


 Name     Count  
Device & app history 3
Ident